In [ ]:
# ========================================================
#   TASK 3: HANDWRITTEN CHARACTER RECOGNITION (CNN MODEL)
#   Single File Project - MNIST / EMNIST Supported
# ========================================================

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# ========================================================
# 1. LOAD DATASET (MNIST DIGITS)
# For EMNIST: use script below (comment/uncomment)
# ========================================================

print("Loading MNIST dataset...")
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape → CNN expects 4D input
X_train = X_train.reshape(-1, 28, 28, 1)
X_test  = X_test.reshape(-1, 28, 28, 1)

# Normalize pixel values
X_train = X_train / 255.0
X_test  = X_test / 255.0

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

print("Dataset Loaded Successfully!")


# ========================================================
# 2. DATA AUGMENTATION
# ========================================================

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.10,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(X_train)


# ========================================================
# 3. BUILD CNN MODEL
# ========================================================

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(10, activation='softmax')  # 10 classes for MNIST digits
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


# ========================================================
# 4. TRAIN MODEL
# ========================================================

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    epochs=10,
    validation_data=(X_test, y_test)
)


# ========================================================
# 5. EVALUATION
# ========================================================

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")


# ========================================================
# 6. SAVE MODEL
# ========================================================

model.save("handwritten_character_model.h5")
print("Model Saved → handwritten_character_model.h5")


In [ ]:
from tensorflow.keras.datasets import mnist
import tensorflow_datasets as tfds

print("Loading EMNIST Letters dataset...")
ds_train, ds_test = tfds.load('emnist/letters', split=['train', 'test'], as_supervised=True)

# Convert to arrays
X_train = np.array([img.numpy() for img, label in ds_train])
y_train = np.array([label.numpy() for img, label in ds_train])

X_test = np.array([img.numpy() for img, label in ds_test])
y_test = np.array([label.numpy() for img, label in ds_test])

# Reshape + Normalize
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0

# Labels start from 1–26 → shift to 0–25
y_train = y_train - 1
y_test = y_test - 1

y_train = to_categorical(y_train, 26)
y_test  = to_categorical(y_test, 26)

print("EMNIST Loaded.")
